<a href="https://colab.research.google.com/github/Juniorffonseca/processo_seletivo/blob/main/teste_processo_seletivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Traga insumos para validação ou refutação das seguintes hipóteses:
1.1. Ao longo do tempo é possível observar uma mudança na preferência de
escolha de medicamento do médico;
1.2. Existem outros fatores que influenciam na tomada de decisão de qual
medicamento o médico irá prescrever (geográfico e/ou especialidade médica
e/ou temporal);
1.3. Existe uma relação sazonal na escolha do médico pelos medicamentos;
1.4. Existe relação entre tornar-se membro da comunidade e/ou ser visitado com
o número de prescrições para cada um dos medicamentos;
2. Faça um relatório para ser acompanhado semanalmente com os principais
indicadores que julgar relevantes e que demonstre o impacto da comunidade nas
prescrições.


In [14]:
# Carregando pacotes
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Importando csvs

url = 'https://raw.githubusercontent.com/Juniorffonseca/processo_seletivo/main/medicos.csv?token=GHSAT0AAAAAAB4FDFCPXWBVCOAUUYJBBUMYY6WRGJQ'
url_2 = 'https://raw.githubusercontent.com/Juniorffonseca/processo_seletivo/main/membros.csv?token=GHSAT0AAAAAAB4FDFCOE72UXJ5I5YR7RSNMY6WRGVQ'
url_3 = 'https://raw.githubusercontent.com/Juniorffonseca/processo_seletivo/main/prescricoes.csv?token=GHSAT0AAAAAAB4FDFCP6OE7MIISLL4K3WK6Y6WRG5A'
url_4 = 'https://raw.githubusercontent.com/Juniorffonseca/processo_seletivo/main/visitados.csv?token=GHSAT0AAAAAAB4FDFCPUI5VULJ7FMNHJ3YMY6WRHEQ'

medicos = pd.read_csv(url, sep = ';', encoding='latin-1')
membros = pd.read_csv(url_2, sep = ';', encoding='latin-1')
prescricoes = pd.read_csv(url_3, sep = ';', encoding='latin-1')
visitados = pd.read_csv(url_4, sep = ';', encoding='latin-1')

In [28]:
#prescricoes['Mes_prescricao'] = pd.to_datetime(prescricoes['Mes_prescricao'], format='%Y%m').dt.strftime('%Y/%m')
prescricoes_agrupado = prescricoes.groupby(['ID_MEDICO', 'Medicamento_prescrito', 'Mes_prescricao'])
prescricoes_agrupado.first()

#parei aqui

prescricoes_agrupado.plot(x='Mes_prescricao',y='Medicamento_prescrito')

KeyError: ignored